In [4]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cosine


In [5]:
from google.colab import files
uploaded = files.upload()

Saving Customers.csv to Customers.csv


In [6]:
from google.colab import files
uploaded = files.upload()

Saving Products.csv to Products.csv


In [7]:
from google.colab import files
uploaded = files.upload()

Saving Transactions.csv to Transactions.csv


In [8]:

customers = pd.read_csv("Customers.csv")
products = pd.read_csv("Products.csv")
transactions = pd.read_csv("Transactions.csv")

In [9]:
data = transactions.merge(customers, on="CustomerID").merge(products, on="ProductID")

In [10]:
customer_product_matrix = data.pivot_table(index='CustomerID', columns='ProductID', values='Quantity', aggfunc='sum', fill_value=0)

In [11]:
similarity_matrix = 1 - np.array([[cosine(customer_product_matrix.loc[i], customer_product_matrix.loc[j])
                                   for j in customer_product_matrix.index] for i in customer_product_matrix.index])

In [12]:
lookalike_dict = {}
for idx, customer in enumerate(customer_product_matrix.index[:20]):
    similar_customers = np.argsort(-similarity_matrix[idx])[1:4]
    lookalike_dict[customer] = [(customer_product_matrix.index[i], similarity_matrix[idx, i]) for i in similar_customers]

In [13]:
lookalike_df = pd.DataFrame([(key, val[0], val[1]) for key, values in lookalike_dict.items() for val in values],
                            columns=["CustomerID", "LookalikeID", "Score"])
lookalike_df.to_csv("Lookalike.csv", index=False)